<div style="direction:rtl;line-height:300%; font-family: Yas">
    <font face="XB Zar" size=5>
        <div style="direction:rtl;line-height:300%; font-family: Yas">
            <font   size=5>
                <div align=center>
                    <font  size=۲۰>
                        <p></p>
                            درس آنالیز داده ها
                    </font>
                    <br>
                    <font>
                        دانشگاه صنعتی شریف - دانشکده مهندسی برق
                    </font>
                    <br>
                    <font color=blue>
                        تمرین دوم 
                    </font>
                    <br>
                     ۱۳۹۹
                </div>
            </font>
        </div>
    </font>
</div>

## Name : Reza Ramezanpour
## Student ID: 95101558

In [1]:
pip install nltk

In [1]:
import numpy as np
import scipy as sci
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer


# 19 is completely random
np.random.seed(19)

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ramaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
X, y = fetch_20newsgroups(remove=("header","footers","quotes"), return_X_y=True)

<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
از تابع
       Tokenizer
        می توانند استفاده کنید تا یک رشته متنی را به 
    لیستی از لغات تبدیل کنید.
    این تابع 
        stop word 
        ها
        و علامت های نگارشی را حذف می کنید.
    و ریشه لغات را جایگزین می‌کند.
    </font>
</div>

In [4]:
def Tokenizer(str_input):
    str_input = str_input.lower()
    words = word_tokenize(str_input)
    #remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    #stem the words
    porter_stemmer = nltk.PorterStemmer()
    words = [porter_stemmer.stem(w) for w in words if w.isalpha()]
    return words

#example:
Tokenizer("I was wondering if anyone out there could enlighten me on ...")

['wonder', 'anyon', 'could', 'enlighten']

# Tokenizing each news

In [5]:
text_words = []

for text in X:
    text_words.append(Tokenizer(text))

# Using TF-IDF for Vectorization 

In [6]:
tfidf = TfidfVectorizer(analyzer=lambda x:[w for w in x])    
tf_idf_text = np.array(tfidf.fit_transform(text_words).todense())



<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
     تابع زیر را تکمیل کنید. باید یک متن از متن های موجود در   
    مجموعه داده 
    newsgroups
    بگیرد و ۱۰ متن مشابه برگرداند.
    در استفاده از هر 
    package 
    ای
        برای
        LSH
        مختار هستید.
    توجه داشته باشید که فقط باید از متون برای تشخیص شباهت
        استفاده کنید و از 
        بردارY 
        (tag)
       که به هر
        متن
        منسوب  
      شده است باید در ارزیابی مدل استفاده کنید  
    </font>
</div>

# Random Hyperplanes LSH 

<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
    این بخش بصورت مفصل در فایل Report_Programming
    توضیح داده شده است.
    </font>
</div>

In [7]:
from scipy.spatial.distance import cosine

# convert a vector to sign(vector)
def binary_conv(row):
    
    return [1 if x >= 0 else 0 for x in row]

# convert a binary vector to int
def bin2dec(row):
    
    return int("".join(str(x) for x in row), 2)

def similar_text(text, list_similar_vector, list_similar_keys, k, text_id):
    
    # holding cosine values
    cosine_val = []
    cosine_key = []
        
    for i in range(len(list_similar_keys)):

        if(list_similar_keys[i] == text_id):
            continue
            
        # cosine sim 
        cosine_sim = 1 - cosine(text, list_similar_vector[i])

        # if this vector is more similar update nearest values

        
        cosine_val.append(cosine_sim)
        cosine_key.append(list_similar_keys[i])
    
    if(len(cosine_key) <= k):
        return cosine_key
    
    else:
        
        k_index = np.argsort(cosine_key)[::-1][:k]
        
        return [cosine_key[ind] for ind in k_index]
        

class LSH:
    def __init__(self, num_planes=4, num_hashes=20): 
      
        
        # minimum is 8 due to question
        self.num_planes = num_planes

        # number of hash tables
        self.num_hashes = num_hashes
        
        # signature matrix
        self.signature_matrix = []
        self.signature_matrix_keys = []
        
        # norm of planes (considering planes passing through origin)
        self.norm_planes = []
        
    def random_hashing(self):
        

        for i in range(self.num_hashes):

            # generating norm of planes
            self.norm_planes.append(np.random.normal(size = (self.num_planes, tf_idf_text.shape[1])))

            # baskets
            baskets_vector = {}
            baskets_key = {}

            # number of baskets
            num_basket = 2 ** self.num_planes 

            # making a list for holding vectors
            for j in range(num_basket):
                baskets_vector[j] = []
                baskets_key[j] = []

            # finding side of each point from hyperplanes
            side_vector = tf_idf_text @ self.norm_planes[i].T

            # convert it to binary
            binary_side = np.apply_along_axis(binary_conv, 1, side_vector)

            # convert binary to decimal as a hash value
            hash_value = np.apply_along_axis(bin2dec, 1, binary_side)

            # assign to baskets
            for j in range(len(hash_value)):
                baskets_vector[hash_value[j]].append(tf_idf_text[j, :])
                baskets_key[hash_value[j]].append(j)

            self.signature_matrix.append(baskets_vector)
            self.signature_matrix_keys.append(baskets_key)

    
    def recommend(self, text_id, search_text,k):
    
       

        # list of similar vectors to each text
        list_similar_vector = []
        list_similar_keys = []

        for i in range(self.num_hashes):

            # finding side of each search word from hyperplanes
            side_vector = search_text @ self.norm_planes[i].T

            # convert it to binary
            binary_side = [1 if x >= 0 else 0 for x in side_vector]

            # convert binary to decimal as a hash value
            hash_value = int("".join(str(x) for x in binary_side), 2) 

            # hash table for i-th hash 
            hash_table = self.signature_matrix[i]
            hash_keys = self.signature_matrix_keys[i]

            # related baskets to word
            basket_vectors = hash_table[hash_value]
            basket_keys = hash_keys[hash_value]


            for j in range(len(basket_keys)):

                if(basket_keys[j] not in list_similar_keys):
                    list_similar_keys.append(basket_keys[j])
                    list_similar_vector.append(basket_vectors[j])


        return similar_text(search_text, list_similar_vector, list_similar_keys, k, text_id)

# 5 number of planes and 20 hash tables
lsh = LSH(5, 20)
    
# hash french pickles into buckets
lsh.random_hashing()

<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
دو معیار شناخته شد برای ارزیابی سیستم ها توصیه گر
<br>    
        mrr (mean reciprocal ranking)
<br>
        map (mean average precision)
<br>
       هستند.
         تابع های مربوط به این دو معیار پیاده سازی کنید.
        در این پیاده سازی فقط می توانید از 
        numpy
        و 
        scipy 
        استفاده کنید.
        برای توضیح بیشتر می توانید از دو لینک زیر کمک بگیرد.
https://en.wikipedia.org/wiki/Mean_reciprocal_rank <br>
http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html
    </font>
</div>       



In [8]:
def mrrk(pred_list,real_list):
    #mean reciprocal ranking at k
    
    # len of predictions
    q = len(real_list)
    
    # holding sum of (1 / rank)
    sum_rank = 0
    
    for i in range(q):
        
        for j in range(10):
            
            # finding index of first match
            if(y[pred_list[i][j]] == real_list[i]):
                sum_rank += (1 / (j + 1))
                break
               
    return sum_rank / q

In [17]:
def apk(pred_list, real_label):
    
    gtp = 0
    ap_val = 0
    
    for i in range(len(pred_list)):
        
        if(y[pred_list[i]] == real_label):
            gtp += 1
            
            ap_val += (gtp / (i + 1))
            
    if gtp != 0 :
        
        return ap_val / gtp
    else:
        return 0
            
            

def mapk(pred_list,real_list):
    #mean average precision at k
    
    # len of predictions
    q = len(real_list)
    
    sum_ap = 0 
    
    for i in range(q):
        
        sum_ap += apk(pred_list[i], real_list[i])
        
    return sum_ap / q

<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
اکنون ۱۰۰ متن دلخواه را انتخاب کرد و برای هر کدام ۱۰ پیشنهاد مشابه ارائه دهید
        و با دو معیار بالا مبتی بر بودن و نبودن در یک 
        category
        سیستم خود را ارزیابی کنید.
    مراقب باشد خود متنی را  که قرار است متون مشابه را برای آن ارائه دهید را 
        filter
        کنید.
    </font>
</div>

# Creating 100 random numbers

In [9]:
random_news_id = np.random.choice(len(tf_idf_text), 100, replace=False)

# Creating list of recommendations

In [10]:
# holding recommendations for each news
pred_list = []

# number of recommendations
k = 10

for ind in random_news_id:
    pred_list.append(lsh.recommend(ind, tf_idf_text[ind], k))

# Creating list of real labels ( categories )

In [11]:
real_list = []

for ind in range(100):
    real_list.append(y[ind])

# MRR 

In [12]:
mrrk(pred_list, real_list)

0.12876190476190474

# MAP

In [18]:
mapk(pred_list, real_list)

0.12055158730158728

# Evaluation

<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
        
همانطور که می بینید با مدل ساده ای که پیاده کردیم، نتیجه خوبی برای این سیستم توصیه گر بدست نیاوردیم که این نشان دهنده این است که از الگوریتم های پیچیده تری باید استفاده کرد.
    </font>
</div>

<div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
              میزان mrr
        کم است و این بدان معنا است که در لیست ده تایی ما به طور متوسط یک تا دو عدد آیتم مرتبط وجود دارد که عدد پایینی است.
    </font>
</div>

 <div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
               همچنین میزان map
        نیز کم است و این بدان معنی است که آیتم هایی که با ارزش بیشتر توصیه می شوند که یعنی در صدر لیست قرار دارند کمتر مرتبط هستند و یا می توان این طور معنی کرد که آیتم های مرتبط جایگاه بالایی در سیستم توصیه گر ندارند.
    </font>
</div>

    
   

 <div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
               در نهایت شاید بهتر باشد از روش های بهتری برای embedding 
        بردارها و نیز روش های بهتری برای hashing
        مثل LSH Forest
        استفاده کنیم.
    </font>
</div>

    
   

# Evaluating My Ranking metrics

 <div style="direction: rtl; line-height: 300%; font-family: Yas">
    <font  size=4>
               در زیر برای اینکه نشان دهم توابعی که برای محاسبه مقادیر mrr
        و map
        زده ام درست هستند از توابع آماده استفاده می کنم که نشان می دهند توابع من درست پیاده سازی شده اند
    </font>
</div>

    
   

In [22]:
recom_list = []

for i in range(len(real_list)):
    vect= []
    
    for j in range(10):
        
        if(y[pred_list[i][j]] == real_list[i]):
                
            vect.append(1)
                
        else:
            vect.append(0)
            
    recom_list.append(vect)

In [24]:
def mean_reciprocal_rank(rs):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [25]:
mean_reciprocal_rank(recom_list)

0.12876190476190477

In [26]:
def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    >>> r = [0, 0, 1]
    >>> precision_at_k(r, 1)
    0.0
    >>> precision_at_k(r, 2)
    0.0
    >>> precision_at_k(r, 3)
    0.33333333333333331
    >>> precision_at_k(r, 4)
    Traceback (most recent call last):
        File "<stdin>", line 1, in ?
    ValueError: Relevance score length < k
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)


def average_precision(r):
    """Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).
    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)


def mean_average_precision(rs):
    """Score is mean average precision
    Relevance is binary (nonzero is relevant).
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
    >>> mean_average_precision(rs)
    0.78333333333333333
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
    >>> mean_average_precision(rs)
    0.39166666666666666
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])

In [27]:
mean_average_precision(recom_list)

0.1205515873015873